In [1]:
import pandas as pd
import numpy as np
import os
import variablen as var
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from string import ascii_uppercase
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import psutil
import sys
import traceback
from fastai.tabular import *

In [2]:
filename = var.dataset_name_1
csv_file     = var.original_df_1
save_dir = var.prepair_df_dir
column = 'AMT_INCOME_TOTAL'
result_dir = var.result_dir
statistics_dir = var.statistics_dir
result_file = 'result'

In [3]:
def split_data(csv_file,filename, column, save_dir):
    df = pd.read_csv(csv_file)
    df = df.dropna() 
    y = df.loc[:, [column]].values
    #y = df.pop(column).to_numpy()
    df = df.drop(columns=[column])
    X = df.loc[: , df.columns].values
    #  X = df.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=0)
    
    X_train = pd.DataFrame(X_train, columns= df.columns)
    X_test  = pd.DataFrame(X_test,  columns= df.columns)
    y_train = pd.DataFrame(y_train, columns= [column])
    y_test  = pd.DataFrame(y_test,  columns= [column])
    
    X_train.to_csv( save_dir  + column + '/X_train' + '.csv'  ,index=False,sep=';', decimal=',')
    X_test.to_csv(  save_dir  + column + '/X_test'  + '.csv'  ,index=False,sep=';', decimal=',')
    y_train.to_csv( save_dir  + column + '/y_train' + '.csv'  ,index=False,sep=';', decimal=',')
    y_test.to_csv(  save_dir  + column + '/y_test'  + '.csv'  ,index=False,sep=';', decimal=',')

In [4]:
# Quelle https://rmisstastic.netlify.app/how-to/python/generate_html/how%20to%20generate%20missing%20values
def ampute_mcar(X_complete, missing_rate):
    # Mask completly at random some values
    M = np.random.binomial(1, missing_rate, size = X_complete.shape)
    X_obs = X_complete.copy()
    np.putmask(X_obs, M, np.nan)
    print('Percentage of newly generated mising values: {}'.\
      #Change Code:     
      format(np.round(np.sum(pd.isna(X_obs))/X_obs.size,3)))
    
    # warning if a full row is missing
    for row in X_obs:
        if np.all(pd.isna(row)):
            warnings.warn('Some row(s) contains only nan values.')
            break

    # warning if a full col is missing
    for col in X_obs.T:
        if np.all(pd.isna(col)):
            warnings.warn('Some col(s) contains only nan values.')
            break
            
    return X_obs

In [5]:
def ampute(column_set):
    for column in column_set:
        for s in var.test_set:
            print(column)
            print(s)
            X_complete = pd.read_csv(save_dir + column + '/' + s + '.csv' , delimiter=";", decimal=",")
            for missing_rate in var.missing_rats:
                X_obs_mcar =  pd.DataFrame(ampute_mcar(X_complete.to_numpy(), missing_rate))
                X_obs_mcar.columns = [X_complete.columns]
                filename = str(np.int_(100 * missing_rate))
                X_obs_mcar.to_csv(save_dir + column + '/' + s + '/' + filename + ".csv" ,index=False,sep=';', decimal=',')

In [6]:
def log_exception(exception: BaseException, expected: bool = True):
    """Prints the passed BaseException to the console, including traceback.

    :param exception: The BaseException to output.
    :param expected: Determines if BaseException was expected.
    """
    output = "[{}] {}: {}".format('EXPECTED' if expected else 'UNEXPECTED', type(exception).__name__, exception)
    print(output)
    exc_type, exc_value, exc_traceback = sys.exc_info()
    traceback.print_tb(exc_traceback)

In [7]:
def create_trans_table(df):
    result = pd.DataFrame()
    for column in df.columns:
        s = df[[column]].sort_values(by=column, ascending=True).drop_duplicates(ignore_index=True)
        result = pd.concat([result,s], axis=1)
            
        #s = df[[column]]
        #s.to_csv(var.prepair_df_dir + col + '/columns/' + column  ,index=False,sep=';', decimal=',',  float_format='%.2f')
        #s = pd.read_csv(var.prepair_df_dir  + col + '/columns/' + column , delimiter=";", decimal=",")
        #result[column] = s[column]
    return result

In [8]:
def transform(df,trans_df,do_classify):
    for column in df.columns:
        changed = pd.Series(data=-1, index=range(df[column].size))
        if not do_classify:
            if type(column) != str:
                for i, v in trans_df.iterrows():
                    change = df[column].ne(v[column])
                    changed = changed.where(change,1)
                change = changed.ne(-1)
                df[column] = df[column].where(change, -1)
                continue
                
        for i, v in trans_df.iterrows():
            change = df[column].ne(v[column])
            df[column] = df[column].where(change, i)
            changed = changed.where(change,1)
        change = changed.ne(-1)
        df[column] = df[column].where(change, -1)
        df[column] = df[column].astype('float64')
    return df

In [9]:
def retransform(df,trans_df, types):
    for column in df.columns:
        
        for i, v in trans_df.iterrows():
            change = df[column].ne(i)
            df[column] = df[column].where(change,v[column])
        change = df[column].ne(-1)
        #df[column] = df[column].astype(types.loc[column])
        df[column] = df[column].where(change, np.nan)
    return df

In [10]:
def correlation_pearson(df,column):
    corr = df.corr(method='pearson').abs().unstack(level=0)
    result = corr[column].drop(index=column)
    return result.idxmax()

In [11]:
def correlation_kendall(df,column):
    corr = df.corr(method='kendall').abs().unstack(level=0)
    result = corr[column].drop(index=column)
    return result.idxmax()

In [12]:
def correlation_spearman(df,column):
    corr = df.corr(method='spearman').abs().unstack(level=0)
    result = corr[column].drop(index=column)
    return result.idxmax()

In [13]:
def lern_model(train_path, dep_var,cat_names,cont_names, df,df_length,df_ende,procs, epoche=5 ):
    data = (TabularList.from_df(df, path=train_path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                            .split_by_idx(valid_idx=list(range(df_length,df_ende )))
                            .label_from_df(cols=dep_var)
                            .databunch(bs=1024))
    learn = tabular_learner(data, layers=[500,200], metrics=root_mean_squared_error)
    learn.fit(epoche, 4e-2)
    learn.save('mini_train')
    learn.export()
    return learn

In [14]:
def means_impute(column_set):

    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        wert = y_train[column].mean()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        for i in y_files:
            df = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            results[i[0:-4]] = df.replace(df,wert)
        results.to_csv( result_dir  + var.methode_1 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [15]:
def mode_impute(column_set):

    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        wert = y_train[column].mode().iloc[0,]
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        for i in y_files:
            df = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            results[i[0:-4]] = df.replace(df,wert)
        results.to_csv( result_dir  + var.methode_2 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [16]:
def median_impute(column_set):

    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        wert = y_train[column].median()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        for i in y_files:
            df = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            results[i[0:-4]] = df.replace(df,wert)
        results.to_csv( result_dir  + var.methode_3 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [17]:
def correlation_1_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        df = X_train.copy()
        df[y_train.columns] = y_train
        df =  df.drop(columns=['ID'])

        trans_df = create_trans_table(df)
        types = df.dtypes 
        df_temp = transform(df.copy(), trans_df,True )
        #############################################
        col_corr = correlation_pearson(df_temp ,y_train.columns[0])
        # modi_df DF mit Ziel und Corr Spalte
        modi_df = df_temp[[col_corr]] 
        modi_df[y_train.columns[0]] = df_temp[y_train.columns[0]]
        modi_df = modi_df.groupby([col_corr])[y_train.columns[0]].median()
        modi_df =  modi_df.reset_index(level=col_corr)
        # hinzufügen Missing Bedingung
        df2 =pd.DataFrame([[np.nan, np.nan]], columns=[col_corr, y_train.columns[0]])
        modi_df = modi_df.append(df2, ignore_index=True)
        
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
             
            df_x = df_x.drop(columns=['ID'])  
            x_test = transform(df_x, trans_df,True )

            for a, v in modi_df.iterrows():
                change = x_test[col_corr].ne(v[col_corr])
                y_test[column] = y_test[column].where(change,v[column])

            results[i[0:-4]] = retransform(y_test, trans_df, types)
        results.to_csv( result_dir  + var.methode_4 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [18]:
def correlation_2_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        df = X_train.copy()
        df[y_train.columns] = y_train
        df =  df.drop(columns=['ID'])

        trans_df = create_trans_table(df)
        types = df.dtypes 
        df_temp = transform(df.copy(), trans_df,True )
        #############################################
        col_corr = correlation_kendall(df_temp ,y_train.columns[0])
        # modi_df DF mit Ziel und Corr Spalte
        modi_df = df_temp[[col_corr]] 
        modi_df[y_train.columns[0]] = df_temp[y_train.columns[0]]
        modi_df = modi_df.groupby([col_corr])[y_train.columns[0]].median()
        modi_df =  modi_df.reset_index(level=col_corr)
        # hinzufügen Missing Bedingung
        df2 =pd.DataFrame([[np.nan, np.nan]], columns=[col_corr, y_train.columns[0]])
        modi_df = modi_df.append(df2, ignore_index=True)
        
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
             
            df_x = df_x.drop(columns=['ID'])  
            x_test = transform(df_x, trans_df,True )

            for a, v in modi_df.iterrows():
                change = x_test[col_corr].ne(v[col_corr])
                y_test[column] = y_test[column].where(change,v[column])

            results[i[0:-4]] = retransform(y_test, trans_df, types)
        results.to_csv( result_dir  + var.methode_5 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [19]:
def correlation_3_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        df = X_train.copy()
        df[y_train.columns] = y_train
        df =  df.drop(columns=['ID'])

        trans_df = create_trans_table(df)
        types = df.dtypes 
        df_temp = transform(df.copy(), trans_df,True )
        #############################################
        col_corr = correlation_spearman(df_temp ,y_train.columns[0])
        # modi_df DF mit Ziel und Corr Spalte
        modi_df = df_temp[[col_corr]] 
        modi_df[y_train.columns[0]] = df_temp[y_train.columns[0]]
        modi_df = modi_df.groupby([col_corr])[y_train.columns[0]].median()
        modi_df =  modi_df.reset_index(level=col_corr)
        # hinzufügen Missing Bedingung
        df2 =pd.DataFrame([[np.nan, np.nan]], columns=[col_corr, y_train.columns[0]])
        modi_df = modi_df.append(df2, ignore_index=True)
        
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
             
            df_x = df_x.drop(columns=['ID'])  
            x_test = transform(df_x, trans_df,True )

            for a, v in modi_df.iterrows():
                change = x_test[col_corr].ne(v[col_corr])
                y_test[column] = y_test[column].where(change,v[column])
                 
            results[i[0:-4]] = retransform(y_test, trans_df, types)
        results.to_csv( result_dir  + var.methode_6 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [20]:
def rondom_forest_classification_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        X_train = X_train.drop(columns=['ID'])
        df = X_train.copy()
        df[y_train.columns] = y_train
        trans_df = create_trans_table(df)
        types = df.dtypes 
        x_temp = transform(X_train.copy(), trans_df, True )
        y_temp = transform(y_train.copy(), trans_df, True )
        ##############################################
        sc = StandardScaler()
        x_temp = sc.fit_transform(x_temp)
        x_temp_test = sc.transform(x_temp)
        #############################################
        best_tree = -1
        best_mean_squared_error = -1
        ##############################################
        for i in [40,50,60,70,80,90,100,110,120,130,140,150,160,170]:
            try:
                classifier = RandomForestClassifier(n_estimators=i, random_state=0)
                classifier.fit(x_temp, y_temp)
            #except Exception as exception:
            except MemoryError as error:
                log_exception(error)
                continue
            #    print(str(i) + ' kann nicht genommen werden. Speicherporbelme')
            ###    continue                
            y_pred = classifier.predict(x_temp_test)
            ###################################
            new_mean_squared_error = metrics.mean_squared_error(y_temp, y_pred)
            print(str(i) + " durch")
            if best_tree == -1 or new_mean_squared_error < best_mean_squared_error:
                best_mean_squared_error = new_mean_squared_error
                best_tree = i 
                print(best_tree)
                print(best_mean_squared_error)
                print("Best Tree " + str(best_tree))
        
        classifier = RandomForestClassifier(n_estimators=best_tree, random_state=0)
        classifier.fit(x_temp, y_temp)
        ######################################
        ######################################
        results = y_test[[column]].copy()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
            
            df_x = df_x.drop(columns=['ID'])
            x_test = transform(df_x, trans_df, True )
            X_test = sc.transform(x_test)
            y_pred =  classifier.predict(X_test)
            
            y_test = pd.DataFrame(data=y_pred, columns=[df_y.columns[0]])
            results[i[0:-4]] = retransform(y_test, trans_df, types)
        results.to_csv( result_dir  + var.methode_7 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')      

In [21]:
def rondom_forest_regression_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        X_train = X_train.drop(columns=['ID'])
        trans_df = create_trans_table(X_train)
        X_train = transform(X_train.copy(), trans_df, False )
        ##############################################
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_train_test = sc.transform(X_train)
        #############################################
        best_tree = -1
        best_mean_squared_error = -1
        ##############################################
        for i in [40,50,60,70,80,90,100,110,120,130,140,150,160,170]:
        #for i in [110]:
            ###################################
            regressor = RandomForestRegressor(n_estimators=i, random_state=0)
            regressor.fit(X_train, y_train)
            y_pred = regressor.predict(X_train_test)
            ###################################
            new_mean_squared_error = metrics.mean_squared_error(y_train, y_pred)
            print(str(i) + " durch")
            if best_tree == -1 or new_mean_squared_error < best_mean_squared_error:
                best_mean_squared_error = new_mean_squared_error
                best_tree = i 
                print(best_tree)
                print(best_mean_squared_error)
                print("Best Tree " + str(best_tree))
        
        regressor = RandomForestRegressor(n_estimators=best_tree, random_state=0)
        regressor.fit(X_train, y_train)
        ######################################
        ######################################
        results = y_test[[column]].copy()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
            
            df_x = df_x.drop(columns=['ID'])
            df_x = transform(df_x.copy(), trans_df, False )
            print(i[0:-4])
            X_test = sc.transform(df_x)
            y_pred =  regressor.predict(X_test)
            results[i[0:-4]] = y_pred
        results.to_csv( result_dir  + var.methode_8 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')      

In [22]:
def neuronal_network_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + '0.csv' , delimiter=";", decimal=",")
        df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + '0.csv' , delimiter=";", decimal=",")
        ############
        ############
        #RANNBATSCHEN
        ########
        #########
        X_train_length =  X_train.shape[0]
        X_train = X_train.drop(columns=['ID'])
        df_x = df_x.drop(columns=['ID'])
        
        X_train = X_train.append(df_x, ignore_index=True)
        y_train = y_train.append(df_y, ignore_index=True)
        X_train[y_train.columns] = y_train
        print(X_train.shape)
        train_path = save_dir + column + '/train/'  
        X_train.to_csv( train_path + 'train' + '.csv' ,index=False,sep=';', decimal=',')
        cat_names = ['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY', 'CNT_CHILDREN', 'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE','FLAG_EMAIL','OCCUPATION_TYPE', 'CNT_FAM_MEMBERS']
        cont_names = ['DAYS_BIRTH', 'DAYS_EMPLOYED']
        procs = [FillMissing, Categorify, Normalize]
        dep_var = column
        var_metric = 'mean_absolute_error'
        ##############################################
        learn = lern_model(train_path,dep_var,cat_names,cont_names, X_train,X_train_length ,X_train.shape[0] ,procs, epoche=1)
        ######################################
        ######################################
        if os.path.exists(statistics_dir + 'statistics.csv'):
            statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=';', decimal=',')
        else:
            statistics = pd.DataFrame()
        columns = np.array(['methode','metric','column', 'misssing', 'Wert'])
        metric = 'mean_absolute_error'
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        ###################################
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' +  '0.csv' , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
            
            df_x = df_x.drop(columns=['ID'])
            df_test = df_x.copy()
            df_test = df_test.fillna(0)
            df_test[df_y.columns] = df_y.copy()
                                    
            data = (TabularList.from_df(df_test, path=train_path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(valid_idx=range(600,1000))
                           .label_from_df(cols=dep_var)
                           .databunch(bs=64))
            print(learn.validate(dl=data.dl(DatasetType.Valid),metrics=[mean_absolute_error])[0])
            print('test')
            values = np.array([var.methode_9,metric, column])
            values = np.append(values, i[0:-4])
            values = np.append(values , learn.validate(dl=data.dl(DatasetType.Valid),metrics=[mean_absolute_error])[0])
            statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)
        
        statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [23]:
def calc_root_mean_squared_error():
    for methode in var.methodes:
        print(methode)
        
        files = os.listdir(result_dir + methode+ '/' )
        print(files)
        for file in files:
            org_column = file[0:-4]
            result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
            if os.path.exists(statistics_dir + 'statistics.csv'):
                statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")
            else:
                statistics = pd.DataFrame()
            
            original = result.loc[: , org_column]
            result = result.drop(columns=[org_column])
            columns = np.array(['methode','type','column', 'misssing', 'Wert'])


            for column in result.columns:
                print(print(learn.validate(dl=data.dl(DatasetType.Valid),metrics=[mean_absolute_error])))
                values = np.array([methode,'root_mean_squared_error', org_column])
                values = np.append(values, [column])
                values = np.append(values , [np.sqrt(metrics.mean_squared_error(original, result[column]))])
                statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)
        

            statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [24]:
def calc_mean_absolute_error():
    for methode in var.methodes:
        files = os.listdir(result_dir + methode+ '/' )
        for file in files:
            org_column = file[0:-4]
            result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
            if os.path.exists(statistics_dir + 'statistics.csv'):
                statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")
            else:
                statistics = pd.DataFrame()

            original = result.loc[: , org_column]
            result = result.drop(columns=[org_column])
            columns = np.array(['methode','type','column', 'misssing', 'Wert'])

        
            for column in result.columns:
                values = np.array([methode,'mean_absolute_error', org_column])
                values = np.append(values, [column])
                values = np.append(values , [metrics.mean_absolute_error(original, result[column])])
                statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)

            statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [25]:
def calc_mean_squared_error():
    for methode in var.methodes:
        files = os.listdir(result_dir + methode+ '/' )
        for file in files:
            org_column = file[0:-4]
            result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
            if os.path.exists(statistics_dir + 'statistics.csv'):
                statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")

            else:
                statistics = pd.DataFrame()
            
            original = result.loc[: , org_column]
            result = result.drop(columns=[org_column])
            columns = np.array(['methode','type','column', 'misssing', 'Wert'])

        

            for column in result.columns:
                values = np.array([methode,'mean_squared_error', org_column])
                values = np.append(values, [column])
                values = np.append(values , [metrics.mean_squared_error(original, result[column])])
                statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)

            statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [26]:
def calc_f1_score():
    for methode in var.methodes:
        files = os.listdir(result_dir + methode+ '/' )
        for file in files:
            org_column = file[0:-4]
            result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
            
            if os.path.exists(statistics_dir + 'statistics.csv'):
                statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")

            else:
                statistics = pd.DataFrame()
            
            original = result.loc[: , org_column]
            #if original.dtypes == float:
            #    original = original.astype(int)
            werte = pd.DataFrame()
            for column in result.columns:
                werte = werte.append(result[column])
            werte = werte.drop_duplicates(ignore_index=True)
            trans_df = create_trans_table(werte)
            original = transform(original,trans_df,True)
            original = original.astype(int)
            result = result.drop(columns=[org_column])
            columns = np.array(['methode','type','column', 'misssing', 'Wert'])

        

            for column in result.columns:
                
                #if result[column].dtypes == float:
                #    result[column] = result[column].astype(int)
                result[column] = transform(result[column],trans_df,True)
                result[column] = result[column].astype(int)
                values = np.array([methode,'f1_score', org_column])
                values = np.append(values, [column])
                values = np.append(values , [metrics.f1_score(original, result[column], average='weighted',labels=np.unique(result[column]))])
                statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)

            statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [ ]:
split_data(csv_file,filename, column, save_dir)

In [ ]:
ampute(var.application_record)

In [ ]:
 means_impute(var.application_record)

In [ ]:
median_impute(var.application_record)

In [ ]:
mode_impute(var.application_record)

In [27]:
correlation_1_impute(var.application_record)

C:\Users\GreenBook\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
correlation_2_impute(var.application_record)

C:\Users\GreenBook\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [29]:
correlation_3_impute(var.application_record)

C:\Users\GreenBook\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
rondom_forest_regression_impute(var.application_record)

In [ ]:
rondom_forest_classification_impute(var.application_record)

correlation_1
['AMT_INCOME_TOTAL.csv']
0.0
0.0
0.0
0.0
0.0


In [ ]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

In [ ]:
def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

In [30]:
neuronal_network_impute(var.application_record)

(304354, 17)


epoch,train_loss,valid_loss,root_mean_squared_error,time
0,52011986944.000000,51122937856.000000,225396.546875,01:03


IndexError: index out of range in self